In [15]:
import pandas as pd
import os
import numpy as np
from PIL import Image
import PIL


In [20]:
# Get the working directory
working_dir = os.getcwd()

folder_paths = []

#Congregating Data

# Adding all items from working directory
for item in os.listdir(working_dir):
    if os.path.isdir(os.path.join(working_dir, item)):
        folder_paths.append(os.path.join(working_dir, item))


def df_maker(dir_path):
    data = []

    for subdir, dirs, files in os.walk(dir_path):
        for file in files:
            if file.endswith('.png'):  # Make sure it's a png file
                hashId, score, category = file[:-4].split('-')  
                # Add the data to the list
                data.append([hashId, score, category])

    # Create a pandas dataframe from the data
    df = pd.DataFrame(data, columns=['hashId', 'score', 'category'])
    return df

dfs = []

for i in folder_paths:
    df = df_maker(i)
    dfs.append(df)

imagedf = pd.concat(dfs)

imagedf["category"].unique()




array(['Drawing', 'Graffiti', 'ArtPorn', 'Art', 'StreetArt',
       'ComicBookArt', 'IDAP', 'Illustration', 'Painting', 'learnart',
       'Calligraphy', 'Watercolor', 'ArtefactPorn', 'ArtCrit'],
      dtype=object)

In [27]:
###Cleaning Data from unusable files
image_dir = 'enter path'
dirlist = ['Drawing', 'Graffiti', 'ArtPorn', 'Art', 'StreetArt',
       'ComicBookArt', 'IDAP', 'Illustration', 'Painting', 'learnart',
       'Calligraphy', 'Watercolor', 'ArtefactPorn', 'ArtCrit']
# Loop through all files in the directory
funni = 0
for i in dirlist:
    for filename in os.listdir(i):
    # Get the full file path
        path = 'enter path'+i
        filepath = os.path.join(path, filename)
        try:
            # Try to open the file with PIL
            with Image.open(filepath) as img:
                # nonsense code
                funni += 1
        except PIL.UnidentifiedImageError:
            # If PIL raises an UnidentifiedImageError, print an error message
            print(f"Unable to identify image file: {filepath}")

Unable to identify image file: /Users/pranavtavildar/Desktop/images/ArtPorn/.DS_Store
Unable to identify image file: /Users/pranavtavildar/Desktop/images/ArtefactPorn/.DS_Store


In [28]:
import torch
from torchvision import datasets, transforms

# Set the path to the root directory containing the image folders
root_folder = 'enter path'

# Define the transforms to be applied to the images
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 pixels
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the image
])

# Load the datasets from the image folders
image_datasets = datasets.ImageFolder(root_folder, transform=data_transforms)

# Create a dictionary to map the class indices to class names
class_names = image_datasets.classes

# Create a PyTorch DataLoader to iterate through the datasets
batch_size = 32
num_workers = 4
data_loader = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size, shuffle=True, num_workers=num_workers)


In [29]:
from torch.utils.data.sampler import SubsetRandomSampler
# Split the dataset into train, validation, and test sets
num_samples = len(image_datasets)
train_split = int(0.8 * num_samples)
val_split = int(0.1 * num_samples)
test_split = num_samples - train_split - val_split

# Define the samplers for each subset of the data
indices = np.arange(num_samples)
np.random.shuffle(indices)
train_indices = indices[:train_split]
val_indices = indices[train_split:train_split+val_split]
test_indices = indices[train_split+val_split:]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
val_loader = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size, sampler=val_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size, sampler=test_sampler, num_workers=num_workers)

In [30]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load a pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Replace the last fully connected layer with a new layer that has the correct number of output classes
num_classes = len(class_names)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Send the model to the device for training
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model for some number of epochs
num_epochs = 10

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)
    
    # Set the model to training mode
    model.train()
    
    # Initialize the running loss and accuracy for the current epoch
    running_loss = 0.0
    running_corrects = 0
    
    # Iterate over the training set
    for inputs, labels in train_loader:
        # Send the data to the device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Update the running loss and accuracy
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)
    
    # Calculate the epoch loss and accuracy
    epoch_loss = running_loss / len(train_indices)
    epoch_acc = running_corrects.double() / len(train_indices)
    
    print(f'Train loss: {epoch_loss:.4f} | Train accuracy: {epoch_acc:.4f}')
    
    # Set the model to evaluation mode
    model.eval()
    
    # Initialize the running loss and accuracy for the validation set
    running_loss = 0.0
    running_corrects = 0
    
    # Iterate over the validation set
    for inputs, labels in val_loader:
        # Send the data to the device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Update the running loss and accuracy
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)
    
    # Calculate the validation loss and accuracy
    val_loss = running_loss / len(val_indices)
    val_acc = running_corrects.double() / len(val_indices)
    
    print(f'Val loss: {val_loss:.4f} | Val accuracy: {val_acc:.4f}')
    print()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10
----------
Train loss: 2.4154 | Train accuracy: 0.2460
Val loss: 3.4044 | Val accuracy: 0.2083

Epoch 2/10
----------
Train loss: 2.0268 | Train accuracy: 0.3389
Val loss: 2.0773 | Val accuracy: 0.3148

Epoch 3/10
----------
Train loss: 1.7962 | Train accuracy: 0.3990
Val loss: 2.1374 | Val accuracy: 0.3148

Epoch 4/10
----------
Train loss: 1.7146 | Train accuracy: 0.4261
Val loss: 2.4361 | Val accuracy: 0.2870

Epoch 5/10
----------
Train loss: 1.5868 | Train accuracy: 0.4752
Val loss: 2.3621 | Val accuracy: 0.3426

Epoch 6/10
----------
Train loss: 1.4177 | Train accuracy: 0.5150
Val loss: 2.4762 | Val accuracy: 0.2963

Epoch 7/10
----------
Train loss: 1.2960 | Train accuracy: 0.5727
Val loss: 2.8539 | Val accuracy: 0.3287

Epoch 8/10
----------
Train loss: 1.0867 | Train accuracy: 0.6386
Val loss: 2.6620 | Val accuracy: 0.3009

Epoch 9/10
----------
Train loss: 0.9254 | Train accuracy: 0.6900
Val loss: 5.5379 | Val accuracy: 0.2269

Epoch 10/10
----------
Train loss: 0.

In [31]:
torch.save(model,'reddit_model.pth')


In [35]:
model = torch.load('reddit_model.pth')

def prediction(image_path):
    # Define the transformation to apply to the input image
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    
    image = Image.open(image_path).convert('RGB')

    # Apply the transformation to the input image
    image_tensor = transform(image)

    # Add a batch dimension to the image tensor
    image_tensor = image_tensor.unsqueeze(0)

    # Put the model in evaluation mode
    model.eval()

    # Make a prediction on the input image
    with torch.no_grad():
        output = model(image_tensor)

    # Get the predicted class index
    predicted_index = output.argmax().item()

    # Print the predicted class label
    class_labels = ['Drawing', 'Graffiti', 'ArtPorn', 'Art', 'StreetArt',
       'ComicBookArt', 'IDAP', 'Illustration', 'Painting', 'learnart',
       'Calligraphy', 'Watercolor', 'ArtefactPorn', 'ArtCrit'] 
    predicted_label = class_labels[predicted_index]
    print('Predicted label:', predicted_label)

### To test
prediction('path to image')

Predicted label: ComicBookArt
